In [1]:
import os
import sys
import numpy as np
from sklearn import metrics

root_dir = "../"
sys.path.append(root_dir)
import configs
import datasets
import models
from ibydmt import get_config, get_dataset, get_predictions, get_test_classes

# from ibydmt.utils.config import get_config
# from ibydmt.utils.data import get_dataset
# from ibydmt.classifiers import get_predictions
# from ibydmt.tester import get_test_classes

config_name = "chexpert"
config = get_config(config_name)

dataset = get_dataset(config, train=False)
label = [target for _, target in dataset.samples]
classes = dataset.classes

test_classes = get_test_classes(config)
test_classes_idx = np.array([classes.index(c) for c in test_classes])

backbone_configs = config.sweep(["data.backbone"])

In [2]:
accuracy = np.zeros((len(backbone_configs), len(test_classes)))

for i, backbone_config in enumerate(backbone_configs):
    df = get_predictions(backbone_config)

    output = df.values[:, 1:]
    prediction = np.argmax(output, axis=-1)
    confusion_matrix = metrics.confusion_matrix(label, prediction, normalize="true")
    backbone_accuracy = np.diag(confusion_matrix)
    backbone_accuracy = backbone_accuracy[test_classes_idx]

    accuracy[i] = backbone_accuracy

    print(f"{backbone_config.data.backbone}:")
    sorted_class_idx = np.argsort(backbone_accuracy)[::-1]
    sorted_class_names = [test_classes[idx] for idx in sorted_class_idx]
    sorted_backbone_accuracy = backbone_accuracy[sorted_class_idx]
    for class_name, class_accuracy in zip(sorted_class_names, sorted_backbone_accuracy):
        print(f"\t {class_name}: {class_accuracy:.2%}")

    print(
        f"{backbone_config.data.backbone} & "
        + " & ".join([f"${a*100:.2f}\\%$" for a in backbone_accuracy])
    )
    print(
        f"Average: {np.mean(backbone_accuracy):.2%} pm {np.std(backbone_accuracy):.2%}"
    )


accuracy_mu = np.mean(accuracy, axis=0)
accuracy_std = np.std(accuracy, axis=0)
print(
    "average & "
    + " & ".join(
        [
            f"${mu*100:.2f}\\% \pm {std*100:.2f}\\%$"
            for mu, std in zip(accuracy_mu, accuracy_std)
        ]
    )
)
# for class_name, mu, std in zip(test_classes, accuracy_mu, accuracy_std):
#     print(f"{class_name}: {mu:.2%} pm {std:.2%}")
# sorted_class_idx = np.argsort(accuracy_mu)[::-1][:k]
# sorted_accuracy = accuracy_mu[sorted_class_idx]
# sorted_std = accuracy_std[sorted_class_idx]
# sorted_class_names = [test_classes[idx] for idx in sorted_class_idx]
# print(f"Top {k} classes:")
# for class_name, class_accuracy, class_std in zip(
#     sorted_class_names, sorted_accuracy, sorted_std
# ):
#     print(f"\t {class_name}: {class_accuracy:.2%} pm {class_std:.2%}")

# test_classes_path = os.path.join(
#     root_dir, "results", config.name.lower(), "test_classes.txt"
# )
# with open(test_classes_path, "w") as f:
#     for class_name in sorted_class_names:
#         f.write(f"{class_name}\n")

Using downloaded and verified file: /tmp/biovil_t_image_model_proj_size_128.pt


/cis/home/jteneggi/miniconda3/envs/cuda118/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CXRBertTokenizer'.
You are using a model of type bert to instantiate a model of type cxr-bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at microsoft/BiomedVLP-BioViL-T were not used when initializing CXRBertModel: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing CXRBertModel from the checkp

cxr-bert-specialized:
	 abnormal: 83.71%
	 normal: 42.39%
cxr-bert-specialized & $42.39\%$ & $83.71\%$
Average: 63.05% pm 20.66%
average & $42.39\% \pm 0.00\%$ & $83.71\% \pm 0.00\%$


In [3]:
print(np.mean(accuracy), np.std(accuracy))

0.6304722187642641 0.2066216440516204
